In [203]:
import tensorflow as tf
from IPython.display import clear_output
tf.keras.utils.set_random_seed(42)
import skdim
import time
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

### Import data and obtain the number of labels:

In [206]:
filename = 'data/dataset1.pkl'

with open(filename, 'rb') as f:
    data_tuples = pickle.load(f)
f.close()

In [208]:
print('Number of Data Points:', len(data_tuples))
class_labels = []
for i in range(len(data_tuples)):
    class_labels.append(data_tuples[i][1])

print('Class Labels:', len(set(class_labels)))

Number of Data Points: 1000
Class Labels: 2


### Perform Estimation

In [242]:
data = pd.DataFrame([i[0] for i in data_tuples])
data = (data - np.min(data)) / (np.max(data) - np.min(data))
labels = pd.DataFrame([i[1] for i in data_tuples])

In [243]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)

In [244]:
ds_tr = tf.data.Dataset.from_tensor_slices(X_train)
ds_te = tf.data.Dataset.from_tensor_slices(X_test)

In [276]:
tf.keras.utils.set_random_seed(42) # extra code – ensures reproducibility on CPU

ID = 35
radius = 5

shape = len(data_tuples[0][0])

input = tf.keras.layers.Input(shape=(shape,))

latent_minus = tf.keras.layers.Dense(max(ID-radius, 1), activation="relu")(input)
latent_id = tf.keras.layers.Dense(ID, activation="relu")(input)
latent_plus = tf.keras.layers.Dense(ID+radius, activation="relu")(input)

output_minus = tf.keras.layers.Dense(shape)(latent_minus)
output_id = tf.keras.layers.Dense(shape)(latent_id)
output_plus = tf.keras.layers.Dense(shape)(latent_plus)

model = tf.keras.Model(inputs=[input],
                       outputs=[output_minus, output_id, output_plus])

model.compile(loss="mse", optimizer="nadam",
              metrics=["RootMeanSquaredError"])
history = model.fit(X_train, X_train, epochs=10,
                         validation_data=(X_test, X_test))

results = []

for i in history.history:
    results.append((i, history.history[i][-1]))

# 'ID - radius' validation loss
(ID-radius), results[8][1]

clear_output(wait=True)

print("Result of {}: {}\n".format(max(ID-radius, 1),results[8][1]))
print("Result of {}: {}\n".format(ID,results[9][1]))
print("Result of {}: {}\n".format(ID+radius,results[10][1]))

c = coefficient(results[8][1], results[9][1], results[10][1])

print("Coefficient: {}\n".format(c))
print("Elbow Calculation: {}\n".format(c/results[9][1]))

Result of 30: 0.008636359125375748

Result of 35: 0.008770311251282692

Result of 40: 0.008800744079053402

Coefficient: 0.18513146787379536

Elbow Calculation: 21.10888229271443

